In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

In [3]:
# Load the data with market stats
df = pd.read_parquet('data/news_feed_with_market_stats.parquet')
df.tail()

,type,start_date,end_date,ticker,count,text,link,model,growth_last_day,weekly_return,market_daily_return,market_weekly_return,growth_above_market
854,individual,2023-08-08,2023-08-14,DJIA,9,U.S. bank lending remained flat in the latest ...,https://pythoninvest.com/tpost/u7h0i2kxy1-week...,None,0.0040,0.007191,0.00575,-0.006356,0.013547
855,individual,2023-08-08,2023-08-14,RIVN,12,"Rivian, the electric vehicle maker, reported i...",https://pythoninvest.com/tpost/u7h0i2kxy1-week...,None,-0.0005,-0.112026,0.00575,-0.006356,-0.105670
856,individual,2023-08-08,2023-08-14,SPY,7,Here's a summary of the news: - Jim Cramer adv...,https://pythoninvest.com/tpost/u7h0i2kxy1-week...,None,0.0055,-0.005769,0.00575,-0.006356,0.000587
857,market_1_day,2023-08-12,2023-08-13,multiple_tickers,228,This week's financial news was characterized b...,https://pythoninvest.com/tpost/u7h0i2kxy1-week...,market news,NaN,-0.003122,-0.00107,-0.003122,NaN
858,market_1_week,2023-08-08,2023-08-14,multiple_tickers,2210,This week's financial news was characterized b...,https://pythoninvest.com/tpost/u7h0i2kxy1-week...,market news,NaN,-0.006356,0.00575,-0.006356,NaN


In [4]:
def create_document(row):
    """Create a Document object from a DataFrame row."""
    content = f"Type: {row['type']}\n"
    content += f"Period: {row['start_date']} to {row['end_date']}\n"
    content += f"Ticker: {row['ticker']}\n"
    content += f"Growth (last day): {row['growth_last_day']:.2%}\n"
    content += f"Weekly Return: {row['weekly_return']:.2%}\n"
    content += f"Market Daily Return: {row['market_daily_return']:.2%}\n"
    content += f"Market Weekly Return: {row['market_weekly_return']:.2%}\n"
    content += f"Growth Above Market: {row['growth_above_market']:.2%}\n"
    content += f"Count: {row['count']}\n"
    content += f"Content: {row['text']}"
    
    return Document(
        page_content=content,
        metadata={
            'type': row['type'],
            'ticker': row['ticker'],
            'link': row['link'],
            'start_date': str(row['start_date']),
            'end_date': str(row['end_date']),
            'weekly_return': row['weekly_return'],
            'market_daily_return': row['market_daily_return'],
            'market_weekly_return': row['market_weekly_return'],
            'growth_above_market': row['growth_above_market']
        }
    )

# Create documents from DataFrame
documents = [create_document(row) for _, row in df.iterrows()]

In [5]:
# TOP 10 tickers - we will ask questions about them later
df.ticker.value_counts().head(31)

ticker
multiple_tickers    268
NVDA                 54
AMZN                 53
TSLA                 51
AAPL                 47
GOOG                 39
SPY                  37
MSFT                 31
DJIA                 24
SMCI                 21
QQQ                  20
PLTR                 16
VZ                   12
DIS                  12
AMD                  11
META                 11
CRWD                  9
CMG                   9
ASML                  8
AVGO                  7
INTC                  7
ACHC                  6
WMT                   5
COST                  5
COIN                  4
CELH                  4
TGT                   4
MU                    4
CRM                   4
ADBE                  4
RIVN                  3
Name: count, dtype: int64

In [6]:
def setup_qa_chain(documents):
    """Set up the QA chain with the given documents."""
    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    
    # Split documents
    splits = text_splitter.split_documents(documents)
    
    # Initialize embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(splits, embeddings)
    
    # Initialize LLM
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    
    # Create prompt template
    prompt = PromptTemplate(
        template="""You are a financial news analyst assistant. Your task is to provide accurate, 
                well-structured responses based on the provided news articles context. Present the 
                information in chronological order, from earliest to most recent events.

                Format each section with a concise header showing period and performance:
                [YYYY-MM-DD..YYYY-MM-DD, +/-X.X% vs market]

                For individual stocks:
                1. Start each section with the period and growth header format shown above
                2. Follow with key developments and context during that period
                3. Include weekly returns comparison (stock vs market) if significant
                4. Explain what drove the performance
                5. If there were weeks with weeks with a significant difference vs. market (5% or more): add TL;DR in 1-2-3 sentences max in the beginning (before all sections)

                For market-wide analysis:
                1. Use the same chronological structure with period headers
                2. Highlight notable sector or stock-specific movements
                3. Include market-wide return metrics when relevant

                Example format:
                [2024-01-01..2024-01-07, +2.3% vs market]
                Key developments and analysis...

                [2024-01-08..2024-01-14, -1.5% vs market]
                Key developments and analysis...

                Keep each section concise and focused. Do not exceed the line length of 80 
                characters to ensure readability.

                Structure your response to tell a compelling story, even without showing sources.
                Focus on chronological progression while maintaining accuracy and including all 
                key metrics.

                USE ONLY FACTS YOU SEE IN THE NEWS, DO NOT HALLUCINATE. If details are missing,
                omit them or state that the information is not available.

                Question: {question}
                Context: {context}

                Answer: Let's analyze this based on the provided information.""",
        input_variables=["context", "question"]
    )
    
    # Create and return chain
    chain = (
        {"context": vectorstore.as_retriever(search_kwargs={"k": 7}), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain, vectorstore.as_retriever(search_kwargs={"k": 7})

# Setup chain
chain, retriever = setup_qa_chain(documents)

In [7]:
def print_sources(sources):
    """Print source documents in a formatted way."""
    print("\nSource Documents:")
    for i, doc in enumerate(sources, 1):
        print(f"\nSource {i}:")
        print(f"Type: {doc.metadata['type']}")
        print(f"Ticker: {doc.metadata['ticker']}")
        print(f"Period: {doc.metadata['start_date']} to {doc.metadata['end_date']}")
        print(f"Link: {doc.metadata['link']}")
        print(f"Weekly Return: {doc.metadata['weekly_return']:.2%}")
        print(f"Market Weekly Return: {doc.metadata['market_weekly_return']:.2%}")
        print(f"Growth Above Market: {doc.metadata['growth_above_market']:.2%}")

def analyze_news(question, show_sources=True):
    """Analyze news based on a question."""
    # Get response
    response = chain.invoke(question)
    sources = retriever.invoke(question)
    
    # Calculate date range from relevant documents
    dates = [(doc.metadata['start_date'], doc.metadata['end_date']) for doc in sources]
    min_date = min(date[0] for date in dates)
    max_date = max(date[1] for date in dates)
    weeks = round((pd.to_datetime(max_date) - pd.to_datetime(min_date)).days / 7)
    
    # Determine if this is a ticker-specific analysis
    ticker_sources = [doc for doc in sources if doc.metadata['type'] == 'individual']
    if ticker_sources:
        # Get the most common ticker from individual sources
        tickers = [doc.metadata['ticker'] for doc in ticker_sources]
        if tickers:
            most_common_ticker = max(set(tickers), key=tickers.count)
            print(f"\nLong term news for {most_common_ticker} in the last {weeks} weeks ({min_date}..{max_date}):\n")
        else:
            print(f"\nAnalysis for the last {weeks} weeks ({min_date}..{max_date}):\n")
    else:
        print(f"\nAnalysis for the last {weeks} weeks ({min_date}..{max_date}):\n")
    
    print(response)
    
    if show_sources:
        print_sources(sources)

In [8]:
# Test with pharma sector query - kinda ok
tech_query = "What are the recent trends in pharma sector news? Focus on growth patterns and significant developments."
analyze_news(tech_query)


Analysis for the last 13 weeks (2024-10-04..2025-01-06 00:00:00):

[2024-10-04..2024-11-04, -1.9% vs market]
The pharma sector saw a shift towards personalized and precision medicine, particularly in drug development and diagnostics. This trend indicates growing investment opportunities in healthcare.

[2024-11-12..2024-12-16, +0.35% vs market]
Regeneron and Sanofi received FDA approval for Dupixent, a significant development in treating chronic lung diseases. This reflects ongoing advancements in the healthcare sector.

[2024-11-28..2025-01-06, +0.076% vs market]
The pharma sector saw advancements in drug pipelines, particularly in obesity and diabetes treatments. These advancements are attracting investor interest.

[2024-11-10..2024-11-11, +5.05% vs market]
The healthcare sector is expected to see growth in areas such as neuroscience, needle-free injectors, and orthobiologics markets. This indicates a positive outlook for the pharma sector.

Source Documents:

Source 1:
Type: marke

In [9]:
# Test with market summary query - wrong
market_query = "Provide a summary of recent market trends in December 2024. Do not add news before December 2024"
analyze_news(market_query)


Long term news for DJIA in the last 15 weeks (2023-12-11..2024-03-26 00:00:00):

[2023-12-11..2023-12-18, +2.56% vs market]
During this week, the U.S. economy added 199,000 jobs in November 2023, exceeding expectations and indicating a stronger labor market. Consumer sentiment in the U.S. surged by 16.1% year-over-year in December, with inflation expectations declining. Technology firms, particularly Alphabet, focused on artificial intelligence (AI) advancements. Despite a shift in market sentiment towards value stocks, Cathie Wood of ARK Invest actively purchased shares in growth companies. Morgan Stanley closely watched several companies within the clean tech sector, including renewable energy, for potential growth trends in 2024.

[2023-12-14..2023-12-25, -0.11% vs market]
The Dow Jones Industrial Average (DJIA) experienced a weekly return of -0.11%, underperforming the market's weekly return of 0.75%. Despite a surge in consumer confidence and a rise in durable-goods orders, the l

In [10]:
# Test with specific company query
company_query = "What are the latest developments and market sentiment for NVDA (NVIDIA)?"
analyze_news(company_query)


Long term news for NVDA in the last 68 weeks (2023-08-25..2024-12-16 00:00:00):

[2023-08-25..2023-09-04, +5.41% vs market]
Nvidia's stock saw a growth of 5.41%, outperforming the market by 2.91%. The company's strong financial performance, partnerships, and growth potential in the AI and semiconductor markets were key factors driving this performance. The company's recent earnings report surpassed expectations, leading analysts to believe that the stock has room to grow further.

[2024-01-05..2024-01-15, +11.43% vs market]
Nvidia's stock experienced a significant growth of 11.43%, outperforming the market by 9.59%. The company's dominance in the AI market and recent announcements of AI-driven solutions at CES 2024 contributed to its positive momentum in the market. Wall Street analysts remained optimistic about the company's growth potential.

[2024-01-22..2024-01-29, +4.71% vs market]
During this period, Nvidia's stock grew by 4.71%, outperforming the market by 3.11%. The company's 

In [11]:
# Test with specific company query (AMZN)
company_query = "What are the latest developments and market sentiment for Amazon (AMZN)?"
analyze_news(company_query)


Long term news for AMZN in the last 64 weeks (2023-09-01..2024-11-25 00:00:00):

[2023-09-01..2023-09-11, +3.61% vs market]
Amazon's stock performance was positive during this period, with a weekly return of 3.61%, significantly outperforming the market. The company's logistics network and web services segment were highlighted as key drivers of its success. Despite potential legal and regulatory concerns, the overall sentiment remained bullish.

[2023-10-31..2023-11-06, +5.30% vs market]
Amazon's stock continued to perform well, with a weekly return of 5.30%. The company's strong performance in e-commerce and cloud computing drove investor interest. CEO Andy Jassy's comments on the company's growth potential further sparked discussions about its future prospects.

[2024-02-27..2024-03-04, +1.63% vs market]
During this period, Amazon's stock grew by 1.63%, slightly outperforming the market. The company's stock growth was compared favorably to Apple's, and there were positive sentiments

In [12]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Tesla (TSLA)?"
analyze_news(company_query)


Long term news for TSLA in the last 55 weeks (2023-12-01..2024-12-23 00:00:00):

[2023-12-01..2023-12-11, +1.77% vs market]
Tesla's stock saw a weekly return of 1.77%, outperforming the market by 0.61%. During this period, discussions revolved around the potential impact of tax credits on Tesla vehicle purchases, varying stock price predictions for 2024, and debates about the future of artificial intelligence in Tesla's vehicles. The Cybertruck generated significant consumer interest, and there were discussions about potential competition in the electric vehicles market.

[2024-04-02..2024-04-08, -1.28% vs market]
Tesla's stock experienced a weekly return of -1.28%, underperforming the market by 0.49%. The company faced significant attention and volatility due to news about the launch of the Tesla robotaxi, increased investor searches, and debates about the company's focus on autonomy versus lower-cost electric vehicles. Analysts were divided on the stock's future, with opinions rangi

In [13]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Apple (AAPL)?"
analyze_news(company_query)


Long term news for AAPL in the last 63 weeks (2023-09-15..2024-12-02 00:00:00):

[2023-09-15..2023-09-25, -1.06% vs market]
During this period, Apple's stock saw a weekly return of -1.06%, performing better than the market which had a weekly return of -2.61%. The key developments included Morgan Stanley analyst Erik Woodring reiterating an "overweight" rating and a $215 price target for Apple due to strong demand for the iPhone 15. However, Apple's FineWoven iPhone cases received criticism for their susceptibility to scratches and premature wear.

[2023-11-24..2023-12-04, -0.19% vs market]
Apple's stock had a weekly return of -0.19%, underperforming the market's weekly return of 0.43%. Despite the recent declines, analysts remained optimistic about Apple's future, citing strong demand for the iPhone 15 and potential for expansion into new services. Some analysts set a price target of $240 for the stock.

[2024-03-05..2024-03-11, -1.34% vs market]
Apple's stock saw a weekly return of -

In [14]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Alphabet/Google (GOOG)?"
analyze_news(company_query)


Long term news for GOOG in the last 63 weeks (2023-09-08..2024-11-25 00:00:00):

[2024-02-05..2024-02-12, +2.62% vs market]
Alphabet Inc. (GOOG) stock was hailed as a top investment choice by various analysts and investors. The company's strong foothold in artificial intelligence (AI) was cited as a reason for its potential long-term dominance in the sector. Despite a recent dip in its stock price following a quarterly earnings report, Alphabet's potential for growth and its massive cash reserves suggest an optimistic outlook for its future performance. Google's AI chatbot "Google Gemini" also made waves in the AI chatbot landscape.

[2024-02-13..2024-02-19, -5.63% vs market]
Alphabet Inc., the parent company of Google, faced challenges and uncertainties in the AI industry, with discussions about potential competition and the company's position as an AI leader. Despite this, investors considered the long-term potential of Alphabet stock, and there was speculation about the possibility

In [15]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Microsoft (MSFT)?"
analyze_news(company_query)


Long term news for MSFT in the last 43 weeks (2024-01-29..2024-11-25 00:00:00):

[2024-01-29..2024-02-05, -0.99% vs market]
Microsoft's stock experienced a decrease in weekly return of -0.99%, underperforming the market by -1.30%. Despite this, the company's earnings report showed a significant increase in cloud revenue, driven by investments in AI. Analysts maintained mostly positive opinions, raising price targets and maintaining buy ratings for the stock. Microsoft's strong performance led it to overtake Apple as the world's largest company.

[2024-02-05..2024-02-12, +2.37% vs market]
Microsoft's stock saw a weekly return of 2.37%, outperforming the market by 0.77%. The company's recent launch of Azure Quantum, a cloud-based platform for quantum programming, was seen as a positive factor. However, discussions around the company's valuation, particularly in comparison to energy companies, were also noted.

[2024-03-26..2024-04-01, -0.97% vs market]
Microsoft's stock experienced a de

In [16]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Supermicro (SMCI)?"
analyze_news(company_query)


Long term news for SMCI in the last 42 weeks (2024-03-05..2024-12-23 00:00:00):

[2024-09-10..2024-09-16, +9.57% vs market]
Super Micro Computer, Inc. (SMCI) faced multiple class action lawsuits alleging violations of federal securities laws, including misrepresentations of financial growth, related party transactions, and compliance with export restrictions. Despite these allegations, the company's stock price increased by 9.57% during this period.

[2024-09-17..2024-09-23, +3.75% vs market]
Despite facing a series of class action lawsuits alleging violations of federal securities laws, SMCI's stock price increased by 3.75% during this period. The lawsuits were based on allegations of accounting irregularities, related party transactions, financial misrepresentations, export violations, and other compliance issues.

[2024-10-01..2024-10-07, +14.65% vs market]
SMCI's stock experienced a significant rebound of up to 18% following an update on robust sales of its server systems, includi

In [17]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for the market tickers (SPY, QQQ, or DJIA)?"
analyze_news(company_query)


Long term news for SPY in the last 49 weeks (2023-12-03..2024-11-11 00:00:00):

[2023-12-14..2023-12-25, +0.92% vs market]
Key developments and analysis:
The period saw a downward revision of the Q3 GDP from 5.2% to 4.9%, amid an influx of $20.8 billion into the S&P 500 ETF Trust in a single day. President Joe Biden's approval rating hit a new low of 34%, and Goldman Sachs revised the S&P 500 index target to 5100. Economists Larry Summers and Mohamed El-Erian shared opinions on potential future economic impacts.

[2024-02-05..2024-02-12, +1.71% vs market]
Key developments and analysis:
Mixed consumer sentiment was reported in economic surveys from the New York Federal Reserve and the National Association for Business Economics. While consumer inflation expectations remained stable, there was growing optimism about household finances, credit access, and the stock market. However, economists expressed unease with the Federal Reserve's monetary policies, viewing them as overly restrictiv

In [18]:
# Test with specific company query Palantir
# Ivan: what are the news on +21% (end of 2023) and +40% (end of 2024) vs. market?
company_query = "What are the latest developments and market sentiment for Palantir (PLTR)?"
analyze_news(company_query)


Long term news for PLTR in the last 61 weeks (2023-10-31..2024-12-30 00:00:00):

[2023-10-31..2023-11-06, +21.43% vs market]
Palantir Technologies Inc. (PLTR) reported solid third-quarter results, leading to a significant rally in its stock. The company's strong performance, particularly in the artificial intelligence sector, generated interest in the investment community. However, some analysts expressed concerns about its growth rate.

[2024-09-10..2024-09-16, +1.98% vs market]
Palantir Technologies experienced a surge in its stock price, with a significant increase of over 100% this year. The company's commercial revenue growth outpaced its government segment, and it was added to the S&P 500 index. Despite strong growth and profitability, some analysts cautioned that the stock may be overvalued.

[2024-10-10..2024-11-11, +40.42% vs market]
Palantir Technologies experienced significant growth driven by its Artificial Intelligence Platform, leading to a 35% surge in its stock price. 

In [19]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Verizon Communications (VZ)?"
analyze_news(company_query)


Long term news for VZ in the last 34 weeks (2023-08-25..2024-04-22 00:00:00):

[2023-08-25..2023-09-04, +2.03% vs market]
Verizon's stock saw a growth of 4.53% during this period, outperforming the market by 2.03%. The company donated $50,000 towards Hurricane Idalia relief and extended its partnership with MetLife Stadium, New York Jets, and New York Giants. They also provided relief to customers impacted by Hurricane Idalia and enhanced the fan experience at various sports stadiums through new partnerships.

[2023-09-01..2023-09-11, -2.01% vs market]
During this period, Verizon's stock declined by 2.64%, underperforming the market by 2.01%. Despite this, the company showcased the impact of 5G on the future of media and entertainment at the IBC2023 event. They also announced an increase in its dividend for the 17th consecutive year and made significant network investments in Austin. Verizon's CEO, Hans Vestberg, was scheduled to speak at the Goldman Sachs Communacopia + Technology Co

In [20]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Disney (DIS)?"
analyze_news(company_query)


Long term news for DIS in the last 26 weeks (2023-08-15..2024-02-12 00:00:00):

[2023-08-15..2023-08-21, -3.30% vs market]
Disney faced legal battles against Governor Ron DeSantis's board of supervisors in Florida. Despite these challenges, analysts remained mostly bullish on Disney stock. The company also implemented cost-cutting strategies and raised prices for Disney+ to boost profits.

[2023-08-22..2023-08-28, -2.00% vs market]
The decline in Disney's stock price was discussed, with some considering it a buying opportunity while others saw it as a trap. Disney's streaming efforts and attendance decline were also analyzed. The company was engaged in a legal battle regarding its special tax district in Florida.

[2023-08-25..2023-09-04, -2.06% vs market]
Disney announced the contestants for the upcoming show 'Golden Bachelor.' However, the company's stock price remained stagnant for the past nine years, and recent performance showed declining profits.

[2023-09-08..2023-09-18, +3.03

In [21]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Advanced Micro Devices (AMD)?"
analyze_news(company_query)


Long term news for AMD in the last 44 weeks (2023-10-31..2024-09-02 00:00:00):

[2023-10-31..2023-11-06, +16.19% vs market]
Advanced Micro Devices (AMD) was trending as the company prepared to sell AI chips worth billions of dollars in the upcoming year. Despite a lukewarm Q4 outlook following strong Q3 results, the company's stock experienced a significant weekly return of 16.19%, outperforming the market by 11.41%. The company's revenue forecast fell short of expectations, but it emphasized its opportunities in the artificial intelligence market.

[2023-12-14..2023-12-25, +0.32% vs market]
AMD's stock attracted significant discussion and attention, with several reports outlining reasons to invest in the company. Factors highlighted included its AI chip business potential, expected growth drivers in 2024, and its recent strong performance. However, the company's weekly return was 0.32%, underperforming the market by 0.43%. AMD also recommended rejection of an unsolicited 'mini-tender

In [22]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for Meta Platforms (META)?"
analyze_news(company_query)


Long term news for META in the last 28 weeks (2023-09-29..2024-04-15 00:00:00):

[2023-09-29..2023-10-09, +3.76% vs market]
Meta Platforms, formerly known as Facebook, received mixed analyst ratings. Despite internal restructuring and a decrease in the size of its Responsible AI team, analysts remained optimistic about the company's growth prospects, particularly in AI and hardware innovations.

[2023-11-24..2023-12-04, -4.39% vs market]
Meta Platforms' stock slumped as CEO Mark Zuckerberg sold his first Meta shares in two years. Despite this, Wall Street analysts remained optimistic about the stock's upside potential. The company also challenged the FTC's 'constitutional authority' in a lawsuit, reflecting ongoing legal issues.

[2024-01-22..2024-01-29, +5.04% vs market]
Analysts provided insights into Meta Platforms' potential performance for the fourth quarter of 2023, focusing on key metrics beyond just top and bottom-line estimates. Despite fluctuations in the stock's value, Wall

In [23]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for CrowdStrike (CRWD)?"
analyze_news(company_query)


Long term news for CRWD in the last 44 weeks (2023-11-24..2024-09-30 00:00:00):

[2023-11-24..2023-12-04, +13.01% vs market]
CrowdStrike (CRWD) stock saw a significant growth of 13.01% during this period, outperforming the market by 12.58%. This was driven by the company's strong financial performance, advanced AI cybersecurity offerings, and ability to navigate a tough economic climate. The company reported a 35% year-over-year revenue increase in its third quarter, beating estimates and indicating a positive outlook for the company. Analysts reiterated positive ratings and increased price targets for the stock.

[2024-03-05..2024-03-11, +1.13% vs market]
During this period, CrowdStrike's stock continued to gain attention and positive reviews from financial analysts, with several firms maintaining or raising their price targets for the company. The company's latest quarterly results surpassed expectations, signaling strong demand and growth potential for the cloud-based cybersecurity

In [24]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for CMG?"
analyze_news(company_query)


Long term news for CMG in the last 26 weeks (2024-06-17..2024-12-16 00:00:00):

[2024-06-17..2024-06-18, +2.08% vs market]
During this period, the market saw a positive return of 2.08%. However, there were no specific developments or news related to CMG.

[2024-09-03..2024-09-09, -3.14% vs market]
The market experienced a downturn during this week, with a negative return of 3.14%. Again, no specific news or developments about CMG were reported during this period.

[2024-11-26..2024-12-02, -2.37% vs market]
CMG's performance was negative during this week, with a return of -2.37% against the market. This underperformance resulted in a growth below the market by -3.67%. Unfortunately, the specific factors driving this performance were not provided.

[2024-11-12..2024-12-16, +0.35% vs market]
In the period from mid-November to mid-December, the market showed resilience with a positive return of 0.35%. The market was influenced by mixed drug efficacy reports, pressures on clean energy valu

In [25]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for ASML?"
analyze_news(company_query)


Long term news for ASML in the last 13 weeks (2024-10-10..2025-01-06 00:00:00):

[2024-10-10..2024-11-11, -5.03% vs market]
ASML, a key player in the semiconductor industry, faced challenges with its stock price dropping due to concerns over revenue guidance, slower recovery in the investment cycle, and export restrictions. Despite disappointing results, some analysts and investors view ASML as a potential investment opportunity due to its long-term growth prospects and technological monopoly in lithography machines for chip manufacturers.

[2024-10-17..2024-11-18, +0.89% vs market]
ASML faced a setback due to weaker-than-expected performance and a class action lawsuit alleging false and misleading statements about the severity of issues facing suppliers, the pace of recovery in the semiconductor industry, and the impact of macroeconomic and industry fluctuations. Despite these challenges, the company remains the sole provider of critical EUV machines required for advanced chip produc

In [26]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for AVGO?"
analyze_news(company_query)


Long term news for AVGO in the last 16 weeks (2024-06-07..2024-09-30 00:00:00):

[2024-06-07..2024-06-18, +23.37% vs market]
Broadcom (AVGO) experienced a significant surge in investor interest and positive analyst recommendations. The company reported strong financial results, beating quarterly sales estimates, and raised its full-year sales outlook by $1 billion. Additionally, Broadcom announced a 10-for-1 stock split, which further excited investors and analysts. The positive earnings, strong AI demand, and stock split announcement have contributed to Broadcom's upward momentum and bullish projections from Wall Street.

[2024-06-18..2024-06-24, -7.94% vs market]
Broadcom provided a preview of VMware Explore 2024, a cloud event scheduled for August 26–29 in Las Vegas. The company's valuation indicates high growth expectations for AVGO stock. Broadcom is experiencing significant growth and attention in the market, with a positive assessment of its cybersecurity solutions, and ongoing

In [27]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for INTC?"
analyze_news(company_query)


Long term news for INTC in the last 47 weeks (2024-01-22..2024-12-16 00:00:00):

[2024-01-22..2024-01-29, -9.08% vs market]
Intel's stock saw a significant decline this week, underperforming the market by 10.68%. This was due to a weaker-than-expected forecast and guidance, despite stronger-than-expected sales and earnings for its fourth quarter. Goldman Sachs maintained its Sell rating for Intel and raised its price target from $34.00 to $39.00. The company's challenges in its core data center segment and increasing competition have also contributed to the negative sentiment.

[2024-01-29..2024-02-05, -2.44% vs market]
Intel's performance continued to lag behind the market, underperforming by 2.74%. The company's AI accelerator pipeline surpassed $2 billion and it won customers for its foundry business. However, concerns arose due to a delay in the opening of a new manufacturing plant in Ohio and disappointment with the company's revenue outlook. Despite these mixed signals, there wa

In [28]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for ACHC?"
analyze_news(company_query)



Long term news for ACHC in the last 10 weeks (2024-10-10..2024-12-16 00:00:00):

[2024-10-29..2024-11-04, -17.79% vs market]
Acadia Healthcare (ACHC) is facing a class-action lawsuit from investors alleging that the company misled them about its business practices, including holding patients against their will, subjecting them to abuse, and deceiving insurance providers. The lawsuits cover the period from February 28, 2020, to October 18, 2024, and have December 16, 2024, as the lead plaintiff deadline. This has led to a significant decline in the company's stock price.

[2024-11-26..2024-12-02, +11.22% vs market]
Investors are reminded of the December 16, 2024 deadline for a securities class action lawsuit against ACHC. The lawsuit alleges that the company made false and/or misleading statements about its business practices, leading to significant legal and financial implications for the company.

[2024-11-12..2024-12-16, -0.78% vs market]
ACHC is facing multiple securities class-act

In [29]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for WMT?"
analyze_news(company_query)


Long term news for WMT in the last 56 weeks (2023-11-21..2024-12-16 00:00:00):

[2023-11-21..2023-11-27, +0.91% vs market]
During this period, Walmart's stock experienced a significant one-day drop of 8% following their fiscal Q3 '24 earnings release. Despite this, the company reported sales growth of 5.2% year-over-year to $160.8 billion, beating the consensus of $159.55 billion, and adjusted EPS above previous year's figures. Concerns were raised about expense overrun and softness at the end of October, with some analysts revising their estimates for revenue and EPS. The CEO's comments about expectations for a 'period of deflation' in the coming months also impacted the stock's performance.

[2023-12-17..2023-12-18, Market-wide analysis]
During this time, market sentiment appeared mixed as positive trends in select sectors were offset by concerns and increased vigilance, particularly in the technology and shipping domains. Investors and analysts were closely watching the Fed's polic

In [30]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for COST?"
analyze_news(company_query)


Long term news for COST in the last 52 weeks (2023-12-17..2024-12-16 00:00:00):

[2024-05-28..2024-06-03, +0.7% vs market]
Costco Wholesale Corporation (COST) reported strong third-quarter earnings during this period, surpassing analyst estimates. Key metrics such as traffic and renewal rates remained robust, leading to positive reactions from several analysts who raised their price targets for COST stock. The company's strong business outlook and opportunities in digital and retail media were cited as reasons for analysts' bullish sentiment. Despite the strong performance, the stock experienced a pullback after the earnings report, with some analysts expressing concerns about its current valuation.

[2024-09-03..2024-09-09, -3.14% vs market]
During this period, there were significant shifts and opportunities within various sectors, impacting both investor sentiment and market outlooks. However, specific developments related to COST were not mentioned.

[2024-11-12..2024-12-16, +0.35%

In [31]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for COIN?"
analyze_news(company_query)


Analysis for the last 54 weeks (2023-12-03..2024-12-16 00:00:00):

[2023-12-03..2023-12-04, N/A vs market]
During this period, analysts were focusing on stocks with potential for strong earnings growth. Investors were paying close attention to dividends, particularly from companies providing high-yield return opportunities. Retail chains announced free cookies as a strategic move to attract customers amidst high inflation rates. In the commodities market, the price of gold reached record highs due to rising geopolitical tensions and speculation about the Federal Reserve’s monetary policy. Notable insider trading activities were observed, including sales and acquisitions by executives in companies like Apple and Coinbase.

[2024-09-03..2024-09-09, N/A vs market]
This period saw significant shifts and opportunities within various sectors, impacting both investor sentiment and market outlooks. However, specific details regarding these shifts and opportunities are not available.

[2024-11

In [32]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for CELH?"
analyze_news(company_query)


Long term news for CELH in the last 18 weeks (2024-09-03..2025-01-06 00:00:00):

[2024-11-12..2024-12-16, +1.24% vs market]
During this period, CELH saw a weekly return of 1.24%, outperforming the market's weekly return of 0.35%. A class action lawsuit was filed against Celsius Holdings, alleging securities fraud between February 29, 2024, and September 4, 2024. Despite this legal action and a recent drop in sales, analysts highlighted potential upside in CELH stock due to discounted valuation and untapped international markets.

[2024-11-17..2024-12-23, -13.54% vs market]
CELH experienced a significant decline in this period, with a weekly return of -13.54% compared to the market's -1.65%. The company faced a declining stock price due to slowing growth and market share loss to rivals. Multiple class action lawsuits were filed against the company, alleging securities fraud and misleading statements about its sales and financial performance. Despite these challenges, some analysts stil

In [33]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for TGT?"
analyze_news(company_query)


Long term news for TGT in the last 70 weeks (2023-08-15..2024-12-16 00:00:00):

[2023-08-15..2023-08-21, -0.41% vs market]
Target Corporation reported better-than-expected Q2 earnings but lowered its profit forecast. Sales declined 4.9% YoY to $24.77 billion, missing analyst consensus. Despite this, the stock rose 5% as strong profit offset the revenue miss and lowered guidance.

[2023-11-14..2023-11-20, +22.08% vs market]
Target posted better-than-expected Q3 financial results, with sales of $25.398 billion and adjusted EPS of $2.10, beating analyst estimates. Despite concerns about inflation and strategic changes, Target's stock surged 17%, gaining investor attention. The company's focus on bottom-line growth and strategic initiatives signaled a potential turnaround.

[2024-05-21..2024-05-27, -9.30% vs market]
Target faced a challenging quarter, reporting weaker-than-expected Q1 earnings, with a decline in sales and adjusted EPS. Despite the turbulent performance, some analysts rais

In [34]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for MU?"
analyze_news(company_query)


Long term news for MU in the last 52 weeks (2023-12-17..2024-12-16 00:00:00):

[2024-03-19..2024-03-25, +24.90% vs market]
Micron Technology (MU) experienced a significant surge in its stock price, with a weekly return of 24.90%, outperforming the market by 23.56%. This growth was driven by better-than-expected quarterly results and an optimistic outlook, leading to multiple analysts raising their price targets for Micron. The company's robust first-quarter revenue and surprise profit, along with strong demand in AI and HBM3, drove its stock to record highs.

[2024-03-26..2024-04-01, +12.89% vs market]
Continuing its positive trend, MU's shares increased by 12.89% in a week, outperforming the market by 12.71%. The company reported better-than-expected financial results and strong third-quarter guidance, leading to positive recommendations from several Wall Street analysts. This impressive growth and optimistic earnings estimates suggest that MU's recent market rally may continue, maki

In [35]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for CRM?"
analyze_news(company_query)


Long term news for CRM in the last 55 weeks (2023-11-24..2024-12-16 00:00:00):

[2024-02-27..2024-03-04, +4.74% vs market]
Salesforce.com (CRM) reported higher-than-expected sales and earnings for the fourth quarter, leading to price target upgrades from several analysts. Despite a disappointing guidance overshadowing its earnings beat, Salesforce's fourth-quarter revenue increased 11% year-over-year to $9.29 billion. The company is well-positioned for the AI boom, according to its Q4 earnings report.

[2024-05-28..2024-06-03, -13.10% vs market]
Salesforce.com experienced a fluctuating stock performance and financial forecasts in response to its first-quarter earnings. The company reported a revenue miss, causing a plunge in its stock price. However, it also reported a solid improvement in cash flow from operations and free cash flow compared to the previous year. Analysts have expressed mixed opinions on the stock.

[2023-11-24..2023-12-04, +11.51% vs market]
Salesforce Inc. reported

In [36]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for ADBE?"
analyze_news(company_query)


Long term news for ADBE in the last 53 weeks (2023-12-08..2024-12-16 00:00:00):

[2023-12-08..2023-12-18, -6.73% vs market]
Adobe (ADBE) experienced a drop in stock price despite strong quarterly results and consecutive growth. This was due to the company's forecast for the current quarter and fiscal 2024 missing consensus estimates. However, some analysts maintained a positive outlook, upgrading to Buy ratings and increasing price targets.

[2024-06-07..2024-06-18, +10.79% vs market]
Adobe's stock surged following impressive quarterly financial updates and upbeat guidance. The company's revenues and earnings beat analysts' estimates, leading to a rise in its stock price. Adobe's investments in Artificial Intelligence (AI) seemed to be yielding results, positioning the company to lead in the AI-infused creativity market.

[2024-06-18..2024-06-24, -0.52% vs market]
Despite a slight dip in the stock price, Adobe continued to show strong performance. The company reported strong revenue g

In [37]:
# Test with specific company query 
company_query = "What are the latest developments and market sentiment for RIVN?"
analyze_news(company_query)


Long term news for RIVN in the last 71 weeks (2023-08-08..2024-12-16 00:00:00):

[2023-08-08..2023-08-14, -11.2% vs market]
Rivian reported its Q2 earnings, revealing revenue of $1.12 billion, beating expectations, and an adjusted loss of $1.08 per share, better than anticipated. The company confirmed the production of 13,992 vehicles in Q2, with 12,640 successful deliveries. Despite the positive earnings, the stock did not see a significant boost, with analysts remaining cautious about its near-term performance.

[2023-09-29..2023-10-09, -20.63% vs market]
Rivian's stock declined following a $1.5 billion convertible debt offering and concerns about management missteps. Analysts lowered their price targets for the stock, citing worries about the timing of the debt offering and its impact on investors. Some analysts believe that the selling of Rivian's stock has been overdone and that the dilution from the debt conversion may not be significant. Rivian is expected to exceed production 